In [24]:
#importation des packages
import pandas as pd
import numpy as np
import re

In [25]:
#importation des données du baromètre
df=pd.read_csv("C:/Users/sylva/OneDrive/Bureau/barometre/donnees_brutes_2021.csv", sep=";")

C:\Users\sylva\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (1,3,68,70) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [26]:
#on enlève les variables inutiles
df=df[['Commune', 'Code_INSEE', 'Commune_cp', 'Dep', 'Cp', 'Nom_dep',
       'Nom_region', 'Malvoyant', 'Taille_commune_2', 'Pratique de la marche',
       'Deplacement_a_pied_vie_courante', 'Deplacement_a_pied_loisir',
       'Lieu_marche', 'Deplacement_a_pied', 'Agencement_voirie',
       'conflits_pietons', 'Conducteurs_respectent',
       'Circulation_vehicules_motorises', 'Deplacements_pied_commune',
       'Marche_securite', 'Securite_rues_services',
       'Securite_rues_residentielles', 'Securite_rejoindre_communes_voisines',
       'Acces_pietons', 'Marche_separe_circulation',
       'Marche_separe_autres_mobilites', 'Traverse_carrefour',
       'Deplacements_enfants_personnes_agees', 'Deplacements_enfants_ecole',
       'Pistes_cyclables', 'Espaces_pietons',
       'Espaces_pietons_libres_empietements', 'Espaces_pietons_entretenus',
       'Espaces_pietons_travaux', 'Signaletique_specifique',
       'Amenagements_confort_marcheurs', 'Trottoirs_eclaires',
       'Commune_ecoute_pietons', 'Commune_effort_marche', 'Situation_pietons',
       'Promotion_deplacement', 'Stationnement_espaces_pietons',
       'Acces_transport_commun', 'Signaletique', 'Information',
       'Resume_condition_marche', 'Endroit_problematique_pieton',
       'Endroit_le_plus_apprecie', 'Criteres_importants', 'Commentaires',
       'Buts_frequents_deplacements', 'Buts_frequents_deplacements_Autre',
       'Lors_de_vos_deplacements', 'Principal_moyen_transport',
       'Raisons_pas_deplacement_pied', 'Permis_de_conduire', 'Possedez-vous',
       'Abonnement_transport_commun', 'Membre_association_marche',
       'Membre_association_defense_pietons', 'Sexe_repondant', 'Age_repondant',
       'Indicateur_ressenti_global', 'Indicateur_securite',
       'Indicateur_confort', 'Indicateur_effort_ville',
       'Indicateur_amenagements_services', 'Indicateur_global',
       'Indicateur_global_declare']]

In [27]:
#on crée une variable géographique Ville avec le numéro de département pour éviter les homonymies
p=re.compile('(\D*) \(([0-9]{2,2})[0-9]{3,3}\)')
def ville_dep(x):
    m=p.match(x)
    return m.group(1)+' ' +m.group(2)
df['Ville']=df['Commune'].apply(ville_dep)

In [28]:
#on importe une base pour obtenir les coordonnées géographiques des communes
ville_france=pd.read_csv("C:/Users/sylva/OneDrive/Bureau/barometre/bases complementaires/villes_france.csv", sep=",")

C:\Users\sylva\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (1,10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [29]:
#on crée sur cette base une colonne Ville équivalente à celle des données, qu'on pourrait fusionner si besoin
def rend_lieu(x) :
    y=str(x)[0]
    return(y)
def rend_lieu2(x) :
    y=str(x)[1]
    return(y)
ville_france['Ville']=ville_france['Ozan']+' '+ville_france['01190'].apply(rend_lieu)+ville_france['01190'].apply(rend_lieu2)

In [30]:
#On recatégorise la variable d'âge
def valeur_age(df1):
    liste=[]
    for i in df1['Age_repondant'] :
        if i=='De 45 à 54 ans':
            liste.append('45-65')
        elif i=='De 65 à 74 ans':
            liste.append('65-')
        elif i=='De 35 à 44 ans':
            liste.append('25-45')
        elif i=='De 26 à 34 ans':
            liste.append('25-45')
        elif i=='75 ans et plus':
            liste.append('65-')
        elif i=='De 55 à 64 ans':
            liste.append('45-65')
        elif i=='De 18 à 25 ans':
            liste.append('-25')
        elif i=='Moins de 18 ans':
            liste.append('-25')
        else :
            liste.append(np.nan)
    df1['Age']=np.array(liste)
valeur_age(df)

In [31]:
#et celle de taille
def valeur_taille(df1):
    liste=[]
    for i in df1['Taille_commune_2']:
        if i=='De 5000 et 19 999 habitants':
            liste.append('5 M-20 M habitants')
        elif i =='De 100 000 et 199 999 habitants':
            liste.append('100 M-200 M habitants')
        elif i =='De 20 000 et 49 999 habitants':
            liste.append('20 M-100 M habitants')
        elif i =='De 50 000 et 99 999 habitants':
            liste.append('20 M-100 M habitants')
        elif i =='200 000 habitants ou plus':
            liste.append('200 M habitants')
        elif i =='Moins de 5000 habitants':
            liste.append('-5 M habitants')
        else :
            liste.append(np.nan)
    df1['Taille']=np.array(liste)
valeur_taille(df)

In [32]:
#on prend en compte une question à réponses multiples : une colonne pour savoir si la réponse est présente ou pas
def habitude(df1):
    possibilite=["Faire mes courses et autres démarches administratives", "Accompagner des enfants à l'école ou autres", 
                 "Me promener, faire du tourisme", "Me rendre sur un lieu de loisirs", "Faire du sport, améliorer ma santé",
                "Me rendre au travail, suivre des études", "Autre"]
    nom_col=['Courses','Ecole_autre', 'Promenade', 'Vers_loisirs', 'Sport','Travail', 'Autre']
    for nombre in range(len(possibilite)):
        p=re.compile(possibilite[nombre])
        liste=[]
        for i in df1['Buts_frequents_deplacements']:
            m=p.search(str(i))
            if m==None:
                liste.append(0)
            else:
                liste.append(1)
        df1[nom_col[nombre]]=np.array(liste)
habitude(df)

In [33]:
#on recatégorise les déplacements loisirs en : 1 si se déplace régulièrement, 0 sinon
def valeur_loisir(df1):
    df1['Loisir']=(df1['Deplacement_a_pied_loisir']=='Tous les jours ou presque').apply(float)+(df1['Deplacement_a_pied_loisir']=='1 à 3 fois par semaine').apply(float)

valeur_loisir(df)

In [34]:
#idem pour vie courante
def valeur_utilitaire(df1):
    df1['Utilitaire']=(df1['Deplacement_a_pied_vie_courante']=='Tous les jours ou presque').apply(float)+(df1['Deplacement_a_pied_vie_courante']=='1 à 3 fois par semaine').apply(float)
valeur_utilitaire(df)

In [35]:
#on transforme les indicateurs en variables numériques
var_indicateur=['Indicateur_ressenti_global', 'Indicateur_securite',
       'Indicateur_confort', 'Indicateur_effort_ville',
       'Indicateur_amenagements_services', 'Indicateur_global', 'Indicateur_global_declare']
def yo_float(x):
    return float(str(x).replace(',','.'))

def indicateurs_nombres(df1, liste_indicateur=var_indicateur):
    for indicateur in var_indicateur :
        if indicateur in df1.columns:
            df1[indicateur]=df1[indicateur].apply(yo_float)
    return df1
df=indicateurs_nombres(df)


In [36]:
#importation de la base note crée dans Lettre par ville, avec les lettres FUB
note=pd.read_csv(r'C:\Users\sylva\OneDrive\Bureau\barometre\bases complementaires\notes.csv', sep=';')

In [37]:
#fusion avec les données
df_complet=df.merge(note, how='left')
""".merge(ville_france[['Ville','4.91667','46.3833']], how='left')"""#rajout des coordonnées géographiques

".merge(ville_france[['Ville','4.91667','46.3833']], how='left')"

In [38]:
df_complet.count()

Commune                           43267
Code_INSEE                        43267
Commune_cp                        43267
Dep                               43267
Cp                                43267
                                  ...  
n_tot_valide_net                  25105
classement_note_globale_classe    25105
note_globale_moy_classe           25105
Climat_Velo_num                   25105
lettre_cycl                       25105
Length: 100, dtype: int64

In [53]:
taux_pauvrete[taux_pauvrete['CODGEO']=='91377']

,CODGEO,Taux_pauvrete,niveau_vie_median_milliers


In [54]:
taux_pauvrete[taux_pauvrete['CODGEO']==91377]

,CODGEO,Taux_pauvrete,niveau_vie_median_milliers
34472,91377,16.0,23.22


In [55]:
#on importe la base INSEE donnant le revenu médian et on la fusionne
taux_pauvrete=pd.read_csv(r'C:\Users\sylva\OneDrive\Bureau\barometre\bases complementaires\base-cc-filosofi-2018_CSV_geo2021\taux_pauvrete.csv', sep=';')
taux_pauvrete_lecture=pd.read_csv(r'C:\Users\sylva\OneDrive\Bureau\barometre\bases complementaires\base-cc-filosofi-2018_CSV_geo2021\meta_cc_filosofi_2018_COM-geo2021.csv', sep=';')
taux_pauvrete['Taux_pauvrete']=taux_pauvrete['TP6018']
taux_pauvrete=taux_pauvrete[taux_pauvrete['TP6018'].notna()]
taux_pauvrete['niveau_vie_median_milliers']=taux_pauvrete['MED18']/1000
taux_pauvrete=taux_pauvrete[['CODGEO','Taux_pauvrete','niveau_vie_median_milliers']]
taux_pauvrete_lecture['CODGEO']=taux_pauvrete_lecture['COD_MOD'].apply(str)
taux_pauvrete['CODGEO']=taux_pauvrete['CODGEO'].apply(str)
df_pauvrete=taux_pauvrete.merge(taux_pauvrete_lecture[['CODGEO','LIB_MOD']])
df_pauvrete

C:\Users\sylva\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,CODGEO,Taux_pauvrete,niveau_vie_median_milliers,LIB_MOD
0,01004,18.0,20.42,Ambérieu-en-Bugey
1,01014,10.0,21.31,Arbent
2,01025,5.0,23.02,Bâgé-Dommartin
3,01031,17.0,19.71,Bellignat
4,01032,7.0,22.05,Béligneux
...,...,...,...,...
4326,97420,37.0,15.84,Sainte-Suzanne
4327,97421,59.0,11.88,Salazie
4328,97422,41.0,14.88,Le Tampon
4329,97423,39.0,15.09,Les Trois-Bassins


In [56]:
#on crée un équivalent pour la ville, puis on fusionne les deux bases
def rend_lieu(x) :
    y=str(x)[0]
    return(y)
def rend_lieu2(x) :
    y=str(x)[1]
    return(y)
df_pauvrete['Ville']=df_pauvrete['LIB_MOD']+' '+df_pauvrete['CODGEO'].apply(rend_lieu)+df_pauvrete['CODGEO'].apply(rend_lieu2)
df_complet=df_complet.merge(df_pauvrete[['Ville','Taux_pauvrete', 'niveau_vie_median_milliers']], how='left')


In [19]:
df_complet['Taux_pauvrete']=df_complet['Taux_pauvrete'].apply(float)

In [20]:
#on crée des indicateurs plus directs : 
df_complet['Ville au taux de pauvrete superieur à la moyenne nationale']=(df_complet['Taux_pauvrete']>14.8).apply(float)
df_complet['Plus revenu median']=(df_complet['niveau_vie_median_milliers']>21.730).apply(float)

In [21]:
#on sélectionne les colonnes qui nous intéressent
df_complet=df_complet[['Nom_dep', 'Taille_commune_2','Taux_pauvrete', 'lettre', 'moyenneIndicateur_global', 
        'lettre_cycl', 'niveau_vie_median_milliers','Loisir', 'Utilitaire',
       'Nom_region', 'Malvoyant', 'Pratique de la marche',
       'Deplacement_a_pied_vie_courante', 'Deplacement_a_pied_loisir',
       'Lieu_marche', 'Deplacement_a_pied', 'Agencement_voirie',
       'conflits_pietons', 'Conducteurs_respectent',
       'Circulation_vehicules_motorises', 'Deplacements_pied_commune',
       'Marche_securite', 'Securite_rues_services',
       'Securite_rues_residentielles', 'Securite_rejoindre_communes_voisines',
       'Acces_pietons', 'Marche_separe_circulation',
       'Marche_separe_autres_mobilites', 'Traverse_carrefour',
       'Deplacements_enfants_personnes_agees', 'Deplacements_enfants_ecole',
       'Pistes_cyclables', 'Espaces_pietons',
       'Espaces_pietons_libres_empietements', 'Espaces_pietons_entretenus',
       'Espaces_pietons_travaux', 'Signaletique_specifique',
       'Amenagements_confort_marcheurs', 'Trottoirs_eclaires',
       'Commune_ecoute_pietons', 'Commune_effort_marche', 'Situation_pietons',
       'Promotion_deplacement', 'Stationnement_espaces_pietons',
       'Acces_transport_commun', 'Signaletique', 'Information',
       'Resume_condition_marche', 'Criteres_importants',
       'Buts_frequents_deplacements', 'Buts_frequents_deplacements_Autre',
       'Lors_de_vos_deplacements', 'Principal_moyen_transport',
       'Raisons_pas_deplacement_pied', 'Permis_de_conduire', 'Possedez-vous',
       'Abonnement_transport_commun', 'Membre_association_marche',
       'Membre_association_defense_pietons', 'Sexe_repondant', 'Age_repondant',
       'Indicateur_ressenti_global', 'Indicateur_securite',
       'Indicateur_confort', 'Indicateur_effort_ville',
       'Indicateur_amenagements_services', 'Indicateur_global',
       'Indicateur_global_declare', 'Ville', 'Age', 'Taille',
       'Courses','Ecole_autre', 'Promenade', 'Vers_loisirs', 'Sport','Travail', 'Autre', 
        'Ville au taux de pauvrete superieur à la moyenne nationale', 'Plus revenu median']]

In [22]:
#on passe les notes sur 20
colonnes=['Deplacement_a_pied','Agencement_voirie',
       'conflits_pietons', 'Conducteurs_respectent',
       'Circulation_vehicules_motorises', 'Deplacements_pied_commune',
       'Marche_securite', 'Securite_rues_services',
       'Securite_rues_residentielles', 'Securite_rejoindre_communes_voisines',
       'Acces_pietons', 'Marche_separe_circulation',
       'Marche_separe_autres_mobilites', 'Traverse_carrefour',
       'Deplacements_enfants_personnes_agees', 'Deplacements_enfants_ecole',
       'Pistes_cyclables', 'Espaces_pietons',
       'Espaces_pietons_libres_empietements', 'Espaces_pietons_entretenus',
       'Espaces_pietons_travaux', 'Signaletique_specifique',
       'Amenagements_confort_marcheurs', 'Trottoirs_eclaires',
       'Commune_ecoute_pietons', 'Commune_effort_marche', 'Situation_pietons',
       'Promotion_deplacement', 'Stationnement_espaces_pietons',
       'Acces_transport_commun', 'Signaletique', 'Information',
       'Resume_condition_marche', 'Indicateur_ressenti_global', 'Indicateur_securite',
       'Indicateur_confort', 'Indicateur_effort_ville',
       'Indicateur_amenagements_services', 'Indicateur_global',
       'Indicateur_global_declare']
for colonne in colonnes :
    df_complet[colonne]=(df_complet[colonne]-1)*4

In [23]:
#on crée les indicatrices des notes supérieures à la moyenne, qui permettront de calculer des pourcentages de satisfaction :
for col in colonnes :
    liste_col=[]
    for i in df_complet[col] :
        if i!=float(i):
            liste_col.append(i)
        else : 
            liste_col.append((1-float((i>10)))*100)#.apply(float))*100)
    df_complet[col+'_pourcent_insat']=np.array(liste_col)

In [24]:
#on rajoute l'intercommunalité d'appartenance
df_inter=pd.read_csv("C:/Users/sylva/OneDrive/Bureau/barometre/bases complementaires/intercommunalite.csv", sep=";", skiprows=5)
df_inter['Ville']=df_inter['LIBGEO']+' '+df_inter['DEP']
df_complet=df_complet.merge(df_inter[['Ville', 'LIBEPCI']], how='left')

In [25]:
#on prend en compte les propositions sur le même modèle que les raisons de déplacement : une colonne par possibilité 
#(3 choix max), 1 si l'a cité, 0 sinon
def proposition(df1):
    possibilite=["Verbaliser davantage le stationnement sur les passages piétons et les trottoirs", 
                 "Réserver l'usage des trottoirs aux déplacements à pied", "Modérer la vitesse des véhicules motorisés dans les lieux fréquentés par les piétons",
                "Constituer un réseau complet de cheminements piétons dans la ville", "Davantage de lieux où les piétons sont prioritaires",
                "plus larges, bien entretenus, sécurisés et désencombrés",
                 "Des actions de communication pour développer le gout et le plaisir de la marche",
                "Faciliter l'accès à pied des transports publics et lieux accueillant du public ",
                "Une signalétique adaptée aux piétons", "Des circulations piétonnes éloignées ou protégées  des zones bruyantes",
                "Une meilleure qualité de l'air", "Un meilleur accès aux transports en commun pour faciliter leur usage par les piétons",
                "Un environnement des cheminements piétons plus agréables : végétation, paysage...",
                "Rendre la marche plus agréable pour les piétons : banc de repos, toilettes, eau potable…",
                "Traiter les points noirs et les coupures urbaines ..."]
    nom_col=['Verbaliser','Reserver', 'Vitesse', 'Compléter', 'Priorite','Trottoir', 'Communication', 'Acces', 'Signaletique proposee',
            'Bruit', 'Air', 'Transport', 'Environnement', 'Agreable', 'Point_noir']
    for nombre in range(len(possibilite)):
        p=re.compile(possibilite[nombre])
        liste=[]
        for i in df1['Criteres_importants']:
            m=p.search(str(i))
            if m==None:
                liste.append(0)
            else:
                liste.append(1)
        df1[nom_col[nombre]]=np.array(liste)
    return df1
proposition(df_complet)

,Nom_dep,Taille_commune_2,Taux_pauvrete,lettre,moyenneIndicateur_global,lettre_cycl,niveau_vie_median_milliers,Loisir,Utilitaire,Nom_region,...,Trottoir,Communication,Acces,Signaletique proposee,Bruit,Air,Transport,Environnement,Agreable,Point_noir
0,Ille-et-Vilaine,De 5000 et 19 999 habitants,6.0,A+,4.726466,C,24.29,0.0,1.0,Bretagne,...,0,0,0,0,0,0,0,0,0,0
1,Ille-et-Vilaine,De 5000 et 19 999 habitants,6.0,A+,4.726466,C,24.29,0.0,1.0,Bretagne,...,1,0,0,0,0,0,0,0,0,0
2,Ille-et-Vilaine,De 5000 et 19 999 habitants,6.0,A+,4.726466,C,24.29,0.0,0.0,Bretagne,...,0,1,1,0,0,0,0,0,0,0
3,Ille-et-Vilaine,De 5000 et 19 999 habitants,6.0,A+,4.726466,C,24.29,1.0,1.0,Bretagne,...,0,0,0,0,0,0,0,0,0,0
4,Ille-et-Vilaine,De 5000 et 19 999 habitants,6.0,A+,4.726466,C,24.29,0.0,1.0,Bretagne,...,1,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48317,Pas-de-Calais,Moins de 5000 habitants,25.0,NaN,NaN,NaN,19.61,1.0,1.0,Hauts-de-France,...,0,0,0,0,0,0,1,0,1,0
48318,Pas-de-Calais,Moins de 5000 habitants,NaN,NaN,NaN,NaN,25.73,0.0,1.0,Hauts-de-France,...,0,1,0,0,0,0,0,0,1,0
48319,Pas-de-Calais,Moins de 5000 habitants,19.0,NaN,NaN,NaN,19.83,0.0,1.0,Hauts-de-France,...,0,1,0,0,0,0,0,0,1,0
48320,Pas-de-Calais,Moins de 5000 habitants,NaN,NaN,NaN,NaN,20.69,0.0,1.0,Hauts-de-France,...,0,1,0,0,0,0,0,0,1,0


In [26]:
#on intègre le taux d'accidentalité, égal à 0 si aucun accident n'a été relevé
df_accident=pd.read_csv("C:/Users/sylva/OneDrive/Bureau/barometre/bases complementaires/accidents.csv", sep=";")
df_accident['nombre_accidents_pietons']=df_accident["nombre d'accidents pietons"]
df_complet=df_complet.merge(df_accident, how='left')
df_complet["nombre_accidents_pietons"] = df_complet["nombre_accidents_pietons"].fillna(0)

In [30]:
#on importe la base ayant la population par commune
df_nom_comm=pd.read_csv("C:/Users/sylva/OneDrive/Bureau/barometre/bases complementaires/population par commune.csv", sep=";", skiprows=10)

C:\Users\sylva\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [31]:
#cas spécial de la Corse et des départements 1 à 9 pris en compte, puis fusion
def prendre1(x) :
    if len(str(x))==4 :
        return(str(0))   
    else:
        return str(x)[0]
def prendre2(x):
    if len(str(x))==4 :
        return str(x)[0]
    elif str(x)[1]=='A' or str(x)[1]=='B':
        return str(0)
    else :
        return str(x)[1]
    
df_nom_comm['Ville']=df_nom_comm['LIBGEO']+' '+df_nom_comm['CODGEO'].apply(prendre1)+df_nom_comm['CODGEO'].apply(prendre2)
df_population=df_nom_comm[['Ville','Population totale']]

In [32]:
#à nouveau, un indicateur plus clair :
df_complet=df_complet.merge(df_population, how='left')
df_complet['pourcent_accident']=(df_complet["nombre_accidents_pietons"]*1000)/df_complet['Population totale']
df_complet['plus_accident']=(df_complet['pourcent_accident']>42784/67000).apply(float)
df_complet[['Ville','Population totale','pourcent_accident', 'plus_accident']]

,Ville,Population totale,pourcent_accident,plus_accident
0,Acigné 35,6814.0,0.146757,0.0
1,Acigné 35,6814.0,0.146757,0.0
2,Acigné 35,6814.0,0.146757,0.0
3,Acigné 35,6814.0,0.146757,0.0
4,Acigné 35,6814.0,0.146757,0.0
...,...,...,...,...
48354,Montreuil 62,1952.0,0.000000,0.0
48355,Montreuil 62,1952.0,0.000000,0.0
48356,Montreuil 62,1952.0,0.000000,0.0
48357,Montreuil 62,1952.0,0.000000,0.0


In [34]:
# créatio de deux indicateurs supplémentaires :
df_complet['Indicateur_marchabilité']=(df_complet['Indicateur_securite']+df_complet['Indicateur_confort']+df_complet['Indicateur_ressenti_global'])/3
df_complet['Indicateur_mise_en_valeur_marche']=(df_complet['Indicateur_effort_ville']+(df_complet['Signaletique_specifique']+df_complet['Acces_transport_commun']+df_complet['Information'])/3)/2

In [35]:
df_complet_marseille=df_complet[df_complet['Ville']=='Marseille 13']

In [36]:
#exportations d'un dataframe prêt à être utilisé !
df_complet.to_csv(r'C:\Users\sylva\OneDrive\Bureau\barometre\tableau 05.csv', index=False, sep=';')

In [40]:
#u=df_complet[df_complet['Ville']=='Saint-Gilles 35'][['4.91667','46.3833']]


Ensuite, des bases de données servant notamment à certains graphiques sur Tableau Public

In [34]:
df_export=df_complet[['Ville','Nom_dep', 'Taille_commune_2', 'lettre', 'moyenneIndicateur_global', 
        'Loisir', 'Utilitaire',
       'Nom_region', 'Malvoyant', 'Pratique de la marche',
       'Deplacement_a_pied_vie_courante', 'Deplacement_a_pied_loisir',
       'Lieu_marche', 'Deplacement_a_pied', 'Agencement_voirie',
       'conflits_pietons', 'Conducteurs_respectent',
       'Circulation_vehicules_motorises', 'Deplacements_pied_commune',
       'Marche_securite', 'Securite_rues_services',
       'Securite_rues_residentielles', 'Securite_rejoindre_communes_voisines',
       'Acces_pietons', 'Marche_separe_circulation',
       'Marche_separe_autres_mobilites', 'Traverse_carrefour',
       'Deplacements_enfants_personnes_agees', 'Deplacements_enfants_ecole',
       'Pistes_cyclables', 'Espaces_pietons',
       'Espaces_pietons_libres_empietements', 'Espaces_pietons_entretenus',
       'Espaces_pietons_travaux', 'Signaletique_specifique',
       'Amenagements_confort_marcheurs', 'Trottoirs_eclaires',
       'Commune_ecoute_pietons', 'Commune_effort_marche', 'Situation_pietons',
       'Promotion_deplacement', 'Stationnement_espaces_pietons',
       'Acces_transport_commun', 'Signaletique', 'Information',
       'Resume_condition_marche', 'Criteres_importants',
       'Buts_frequents_deplacements', 'Buts_frequents_deplacements_Autre',
       'Lors_de_vos_deplacements', 'Principal_moyen_transport',
       'Raisons_pas_deplacement_pied', 'Permis_de_conduire', 'Possedez-vous',
       'Abonnement_transport_commun', 'Membre_association_marche',
       'Membre_association_defense_pietons', 'Sexe_repondant', 'Age_repondant',
       'Indicateur_ressenti_global', 'Indicateur_securite',
       'Indicateur_confort', 'Indicateur_effort_ville',
       'Indicateur_amenagements_services', 'Indicateur_global',
       'Indicateur_global_declare',  'Age', 'Taille',
       'Courses','Ecole_autre', 'Promenade', 'Vers_loisirs', 'Sport','Travail', 'Autre']]

In [38]:
df_export.to_csv(r'C:\Users\sylva\OneDrive\Bureau\barometre\donnees sur 20.csv', index=False, sep=';')

saint-gilles -1.83333	48.15
saint-nazaire -2.2	47.2833
    massy 2,28333 48,7333
    montreuil 2,43333	48,8667
 st marcellin   4,16667	45,4833
chatillon 2,28333	48,8
valence 4,9	44,9333
Sceaux 92 2,28333	48,7833
Ombree 47,44   1,10

Préparation de la base toutes les moyennes

In [43]:
df_complet
var_note =['Deplacement_a_pied','Agencement_voirie', 'conflits_pietons', 'Conducteurs_respectent',
       'Circulation_vehicules_motorises', 'Deplacements_pied_commune',
       'Marche_securite', 'Securite_rues_services',
       'Securite_rues_residentielles', 'Securite_rejoindre_communes_voisines',
       'Acces_pietons', 'Marche_separe_circulation',
       'Marche_separe_autres_mobilites', 'Traverse_carrefour',
       'Deplacements_enfants_personnes_agees', 'Deplacements_enfants_ecole',
       'Pistes_cyclables', 'Espaces_pietons',
       'Espaces_pietons_libres_empietements', 'Espaces_pietons_entretenus',
       'Espaces_pietons_travaux', 'Signaletique_specifique',
       'Amenagements_confort_marcheurs', 'Trottoirs_eclaires',
       'Commune_ecoute_pietons', 'Commune_effort_marche', 'Situation_pietons',
       'Promotion_deplacement', 'Stationnement_espaces_pietons',
       'Acces_transport_commun', 'Signaletique', 'Information', 'Resume_condition_marche','Indicateur_ressenti_global', 'Indicateur_securite',
       'Indicateur_confort', 'Indicateur_effort_ville',
       'Indicateur_amenagements_services', 'Indicateur_global']

In [44]:
liste_question=["Dans cette commune, se déplacer à pied est… (très désagréable | très agréable)",
"L'agencement de la voirie (trottoirs, carrefours, traversées de rue…) me permet de me déplacer partout de manière aisée… (pas du tout | tout à fait)",
"Selon moi les conflits entre piétons et autres mobilités actives sans moteur (vélo, trottinettes, skates, rollers sans moteur électrique…) sont… très fréquents| très rares",
"Les conducteurs de véhicules motorisés respectent les marcheurs et piétons… pas du tout | tout à fait",
"Selon moi la circulation des véhicules motorisés (motos et deux roues, voitures, camions) est… insupportable | pas du tout gênante",
"Dans cette commune les déplacements à pied… (concernent une minorité d’habitants | concernent la quasi-totalité des habitants)",
"En général, dans cette commune, quand je me déplace à pied, je me sens… (en danger| en sécurité)",
"Je peux marcher en sécurité dans les rues commerçantes, pour accéder aux services, aux transports… (pas du tout | tout à fait)",
"Je peux marcher en sécurité dans les rues résidentielles… (pas du tout | tout à fait)",
"Je peux rejoindre les communes voisines en sécurité… (pas du tout | tout à fait)",
"Dans cette commune, les accès piétons des abords de gare ou de transport en commun sont… (très dangereux | très sûrs)",
"A pied être séparé de la circulation motorisée est… (pas du tout important | très important)",
"A pied être séparé des autres mobilités actives (vélos, trottinettes…) est… (pas du tout important | très important)",
"Dans cette commune traverser  les carrefours ou contourner les  rond-point est… (toujours dangereux | jamais dangereux)",
"Pour les enfants, les personnes âgées et les personnes à mobilité réduite se déplacer à pied dans ma commune est… (très dangereux | très sûr)",
"Pour les enfants en âge d’aller à l’école à pied, marcher dans ma commune  est… (très dangereux | très sûr)",
"Dans cette commune,  les aménagements cyclables sont  pour les piétons… (toujours dangereux | jamais dangereux)",
"Dans cette commune les espaces réservés aux piétons sont larges, leur surface est plane et facillite les déplacements… (pas du tout | toujours)",
"Dans cette commune les trottoirs et les espaces réservés aux piétons sont  libres d'empiètements (étalages, terrasses…), d'obstacles (poubelles, poteaux, voitures garées…)… (pas du tout libres et respectés | tout à fait libre et respectés)",
"Dans cette commune les espaces destinés  aux piétons sont bien entretenus (planéité du sol, neige, feuilles mortes…)… (pas du tout entretenus | très bien entretenus)",
"Dans cette commune, lors de travaux sur les espaces réservés aux piétons, une solution alternative et  sûre est proposée… (jamais | toujours)",
"Dans cette commune une signalétique spécifique favorise les déplacements des piétons… (pas du tout | tout à fait)",
"Dans cette commune des aménagements  assurent le confort des marcheurs (toilettes, bancs de repos, lieux de sociabilité pour se tenir à plusieurs sans gêner les autres : placettes, distributeurs d'eau potable... )… (jamais | très souvent)",
"Dans cette commune les trottoirs et circulations piétonnes la nuit sont… (très mal éclairés | très bien éclairés)",
"Cette commune est à l'écoute des besoins des marcheurs et des piétons… (pas du tout à l’écoute | tout à fait à l’écoute)",
"Les efforts faits en faveur de la marche dans ma commune sont… (inexistants | très importants)",
"Dans cette commune, ces deux dernières années la situation des piétons s'est… (Grandement détériorée | Grandement améliorée)",
"Dans cette commune, les actions de promotion des déplacements à pied  sont… (inexistantes | très importantes)",
"Dans cette commune le stationnement des véhicules motorisés sur les cheminements piétons est… (Très fréquent | Très rare)",
"Dans cette commune, comment les piétons peuvent-ils accéder aux transports en commun ?... (Très difficilement | Très facilement)",
"Dans cette commune des cheminements piétons spécifiques sont balisés :  signalétique spécifique piétonne pour accéder aux lieux publics, circuits touristiques, sentiers itinéraires de promenade, parcours marche santé… (Jamais | Toujours)",
"Dans cette commune des brochures, plans, panneaux d'information, guides, topoguides, applications sont mis à disposition des piétons pour faciliter leurs déplacements… (Jamais | Toujours)",
"En résumé, je dirais que dans cette commune les conditions pour marcher   sont… (Très mauvaises | Très bonnes)", 'Indicateur ressenti global','Indicateur securite',
       'Indicateur confort', 'Indicateur effort ville',
       'Indicateur amenagements services', 'Indicateur global']


In [57]:
df_complet['Age'].unique()

array(['45-65', '65-', '25-45', '-25'], dtype=object)

In [59]:
df_moyenne=pd.DataFrame([i for i in range(39)])
df_pur_pieton=df_complet[df_complet['Lors_de_vos_deplacements']=='Un marcheur, un piéton']
df_pmr=df_complet[df_complet['Lors_de_vos_deplacements']=='Une personne a mobilité réduite']
df_jeunes=df_complet[df_complet['Age']=='-25']
df_randonneurs=df_complet[df_complet['Membre_association_marche']=='Oui']
liste_nom=[]
liste_valeur=[]
liste_pourcentage=[]
liste_pourcent_pieton=[]
liste_pmr=[]
liste_jeunes=[]
liste_randonneurs=[]
for i in var_note :
    liste_nom.append(i)
    liste_valeur.append(np.mean(df_complet[i]))
    liste_pourcentage.append(np.mean(df_complet[i+'_pourcent_insat']))
    liste_pourcent_pieton.append(np.mean(df_pur_pieton[i+'_pourcent_insat']))
    liste_pmr.append(np.mean(df_pmr[i+'_pourcent_insat']))
    liste_jeunes.append(np.mean(df_jeunes[i+'_pourcent_insat']))
    liste_randonneurs.append(np.mean(df_randonneurs[i+'_pourcent_insat']))

df_moyenne['Nom question']=np.array(liste_nom)
df_moyenne['Nom_question_pourcent']=df_moyenne['Nom question']+'_pourcent_insat'
df_moyenne['Moyenne']=np.array(liste_valeur)
df_moyenne['Pourcent']=np.array(liste_pourcentage)
df_moyenne['Intitulé question']=np.array(liste_question)
df_moyenne['Nom question']=df_moyenne['Nom question'].replace(r'_', ' ', regex=True)
df_moyenne['Nom_question_pourcent']=df_moyenne['Nom_question_pourcent'].replace('_', ' ', regex=True)
df_moyenne['Pourcent_pieton']=np.array(liste_pourcent_pieton)
df_moyenne['Pourcent_pmr']=np.array(liste_pmr)
df_moyenne['Pourcent_jeunes']=np.array(liste_jeunes)
df_moyenne['Pourcent_randonneurs']=np.array(liste_randonneurs)
df_moyenne=df_moyenne[['Nom question', 'Nom_question_pourcent','Moyenne', 'Pourcent', 'Intitulé question', 'Pourcent_pieton', 'Pourcent_pmr',
                      'Pourcent_jeunes', 'Pourcent_randonneurs']]


In [60]:
df_moyenne

,Nom question,Nom_question_pourcent,Moyenne,Pourcent,Intitulé question,Pourcent_pieton,Pourcent_pmr,Pourcent_jeunes,Pourcent_randonneurs
0,Deplacement a pied,Deplacement a pied pourcent insat,11.026529,40.759369,"Dans cette commune, se déplacer à pied est… (t...",39.123221,61.625282,33.202870,29.407073
1,Agencement voirie,Agencement voirie pourcent insat,9.580218,52.596734,"L'agencement de la voirie (trottoirs, carrefou...",50.916950,77.426637,43.639922,40.600904
2,conflits pietons,conflits pietons pourcent insat,10.531605,45.531746,Selon moi les conflits entre piétons et autres...,46.476290,65.237020,48.401826,32.358415
3,Conducteurs respectent,Conducteurs respectent pourcent insat,9.358256,52.758153,Les conducteurs de véhicules motorisés respect...,50.768492,64.559819,52.772342,37.374812
4,Circulation vehicules motorises,Circulation vehicules motorises pourcent insat,8.710991,59.476910,Selon moi la circulation des véhicules motoris...,57.056152,67.268623,57.599478,45.058938
5,Deplacements pied commune,Deplacements pied commune pourcent insat,9.618584,55.125626,Dans cette commune les déplacements à pied… (c...,53.886123,54.401806,41.617743,59.461136
6,Marche securite,Marche securite pourcent insat,11.331212,37.743882,"En général, dans cette commune, quand je me dé...",37.381015,63.431151,30.658839,26.331649
7,Securite rues services,Securite rues services pourcent insat,12.514481,30.398166,Je peux marcher en sécurité dans les rues comm...,30.735307,56.207675,21.852577,21.935655
8,Securite rues residentielles,Securite rues residentielles pourcent insat,12.438591,30.728021,Je peux marcher en sécurité dans les rues rési...,29.700463,52.144470,24.722766,19.817424
9,Securite rejoindre communes voisines,Securite rejoindre communes voisines pourcent ...,8.384878,60.085154,Je peux rejoindre les communes voisines en séc...,56.147935,67.494357,60.273973,46.202251


In [61]:
df_moyenne.to_csv(r'C:\Users\sylva\OneDrive\Bureau\barometre\moyenne 12.csv', index=False, sep=';', header=True)

Dataframe preconisations

In [40]:
prop=['Verbaliser',
       'Reserver', 'Vitesse', 'Compléter', 'Priorite', 'Trottoir',
       'Communication', 'Acces', 'Bruit', 'Air', 'Transport', 'Environnement',
       'Agreable', 'Point_noir', 'Signaletique proposee']

In [41]:
vrai_nom=["Verbaliser davantage le stationnement sur les passages piétons et les trottoirs", 
                 "Réserver l'usage des trottoirs aux déplacements à pied", 
 "Modérer la vitesse des véhicules motorisés dans les lieux fréquentés par les piétons",
                "Constituer un réseau complet de cheminements piétons dans la ville", 
 "Davantage de lieux où les piétons sont prioritaires",
                "Des cheminements piétons (trottoirs) plus larges, bien entretenus, sécurisés et désencombrés",
                 "Des actions de communication pour développer le gout et le plaisir de la marche",
                "Faciliter l'accès à pied des transports publics et lieux accueillant du public (lieux publics, commerces…)",
                
 "Des circulations piétonnes éloignées ou protégées  des zones bruyantes",
                "Une meilleure qualité de l'air", 
 "Un meilleur accès aux transports en commun pour faciliter leur usage par les piétons",
                "Un environnement des cheminements piétons plus agréables : végétation, paysage...",
                "Rendre la marche plus agréable pour les piétons : banc de repos, toilettes, eau potable…",
                 "Traiter les points noirs et les coupures urbaines (carrefours, traversées de voies  qui obligent à de larges détours)...",
         "Une signalétique adaptée aux piétons"]

In [45]:

def proposition (df_preconi):
    df_prop=pd.DataFrame([i for i in range(15)])
    liste_nom=[]
    liste_valeur=[]
    liste_pourcentage=[]
    for i in prop :
        liste_nom.append(i)
        liste_valeur.append(np.mean(df_preconi[i]))
        #liste_pourcentage.append(np.mean(df_complet[i+'_pourcent_insat']))
    df_prop['Nom question']=np.array(liste_nom)
    df_prop['Moyenne']=np.array(liste_valeur)
    df_prop['Intitule question']=np.array(vrai_nom)
    #df_moyenne['Pourcent']=np.array(liste_pourcentage)
    df_prop=df_prop[['Nom question','Moyenne', 'Intitule question']]
    return df_prop.sort_values('Moyenne', ascending=False)
df_precon=df_complet[df_complet['Lors_de_vos_deplacements']=='Une personne a mobilité réduite']
df_preconisation=proposition(df_precon)
df_preconisation

,Nom question,Moyenne,Intitule question
5,Trottoir,0.497951,Des cheminements piétons (trottoirs) plus larg...
0,Verbaliser,0.385246,Verbaliser davantage le stationnement sur les ...
1,Reserver,0.366803,Réserver l'usage des trottoirs aux déplacement...
12,Agreable,0.303279,Rendre la marche plus agréable pour les piéton...
13,Point_noir,0.215164,Traiter les points noirs et les coupures urbai...
2,Vitesse,0.206967,Modérer la vitesse des véhicules motorisés dan...
3,Compléter,0.202869,Constituer un réseau complet de cheminements p...
4,Priorite,0.145492,Davantage de lieux où les piétons sont priorit...
11,Environnement,0.116803,Un environnement des cheminements piétons plus...
7,Acces,0.092213,Faciliter l'accès à pied des transports public...


In [48]:
df_pieton=df_complet[df_complet['Lors_de_vos_deplacements']=="Un cycliste ou un utilisateur d'engin de déplacement personnel (vélo, trottinette, skate, roller…)"]
proposition(df_pieton)

,Nom question,Moyenne,Intitule question
5,Trottoir,0.412748,Des cheminements piétons (trottoirs) plus larg...
0,Verbaliser,0.363708,Verbaliser davantage le stationnement sur les ...
2,Vitesse,0.350578,Modérer la vitesse des véhicules motorisés dan...
13,Point_noir,0.313769,Traiter les points noirs et les coupures urbai...
11,Environnement,0.251262,Un environnement des cheminements piétons plus...
1,Reserver,0.224217,Réserver l'usage des trottoirs aux déplacement...
3,Compléter,0.223095,Constituer un réseau complet de cheminements p...
4,Priorite,0.190327,Davantage de lieux où les piétons sont priorit...
12,Agreable,0.157783,Rendre la marche plus agréable pour les piéton...
9,Air,0.124228,Une meilleure qualité de l'air


In [53]:
df_pieton=df_complet[df_complet['plus_accident']==0]
proposition(df_pieton)

,Nom question,Moyenne,Intitule question
5,Trottoir,0.387062,Des cheminements piétons (trottoirs) plus larg...
2,Vitesse,0.295710,Modérer la vitesse des véhicules motorisés dan...
3,Compléter,0.292687,Constituer un réseau complet de cheminements p...
12,Agreable,0.242207,Rendre la marche plus agréable pour les piéton...
1,Reserver,0.236478,Réserver l'usage des trottoirs aux déplacement...
0,Verbaliser,0.225515,Verbaliser davantage le stationnement sur les ...
13,Point_noir,0.225335,Traiter les points noirs et les coupures urbai...
6,Communication,0.206433,Des actions de communication pour développer l...
11,Environnement,0.199170,Un environnement des cheminements piétons plus...
4,Priorite,0.142644,Davantage de lieux où les piétons sont priorit...


In [54]:
df_pieton=df_complet[df_complet['plus_accident']==1]
proposition(df_pieton)

,Nom question,Moyenne,Intitule question
5,Trottoir,0.424863,Des cheminements piétons (trottoirs) plus larg...
1,Reserver,0.332392,Réserver l'usage des trottoirs aux déplacement...
0,Verbaliser,0.324145,Verbaliser davantage le stationnement sur les ...
13,Point_noir,0.254505,Traiter les points noirs et les coupures urbai...
11,Environnement,0.252291,Un environnement des cheminements piétons plus...
2,Vitesse,0.245266,Modérer la vitesse des véhicules motorisés dan...
12,Agreable,0.218082,Rendre la marche plus agréable pour les piéton...
3,Compléter,0.207544,Constituer un réseau complet de cheminements p...
4,Priorite,0.179215,Davantage de lieux où les piétons sont priorit...
9,Air,0.126489,Une meilleure qualité de l'air


df_marcheur=df_complet[df_complet['Membre_association_marche']=='Oui']
proposition_marcheur=proposition(df_marcheur)
df_non_marcheur=df_complet[df_complet['Membre_association_marche']=='Non']
proposition_non_marcheur=proposition(df_non_marcheur)
proposition_marcheur.to_csv(r'C:\Users\sylva\OneDrive\Bureau\barometre\preconisationmarcheur.csv', index=False, sep=';')
proposition_non_marcheur.to_csv(r'C:\Users\sylva\OneDrive\Bureau\barometre\preconisationnonmarcheur.csv', index=False, sep=';')

In [62]:
df_malv=df_complet[df_complet['Malvoyant']=='Oui']
proposition_malv=proposition(df_malv)
proposition_malv

,Nom question,Moyenne,Intitule question
5,Trottoir,0.426540,Des cheminements piétons (trottoirs) plus larg...
1,Reserver,0.334913,Réserver l'usage des trottoirs aux déplacement...
0,Verbaliser,0.292259,Verbaliser davantage le stationnement sur les ...
12,Agreable,0.252765,Rendre la marche plus agréable pour les piéton...
2,Vitesse,0.233807,Modérer la vitesse des véhicules motorisés dan...
3,Compléter,0.213270,Constituer un réseau complet de cheminements p...
13,Point_noir,0.194313,Traiter les points noirs et les coupures urbai...
11,Environnement,0.184834,Un environnement des cheminements piétons plus...
4,Priorite,0.151659,Davantage de lieux où les piétons sont priorit...
14,Signaletique proposee,0.105845,Une signalétique adaptée aux piétons


In [63]:
df_complet.columns[70:]

Index(['Taille', 'Courses', 'Ecole_autre', 'Promenade', 'Vers_loisirs',
       'Sport', 'Travail', 'Autre',
       'Ville au taux de pauvrete superieur à la moyenne nationale',
       'Plus revenu median', 'Deplacement_a_pied_pourcent_insat',
       'Agencement_voirie_pourcent_insat', 'conflits_pietons_pourcent_insat',
       'Conducteurs_respectent_pourcent_insat',
       'Circulation_vehicules_motorises_pourcent_insat',
       'Deplacements_pied_commune_pourcent_insat',
       'Marche_securite_pourcent_insat',
       'Securite_rues_services_pourcent_insat',
       'Securite_rues_residentielles_pourcent_insat',
       'Securite_rejoindre_communes_voisines_pourcent_insat',
       'Acces_pietons_pourcent_insat',
       'Marche_separe_circulation_pourcent_insat',
       'Marche_separe_autres_mobilites_pourcent_insat',
       'Traverse_carrefour_pourcent_insat',
       'Deplacements_enfants_personnes_agees_pourcent_insat',
       'Deplacements_enfants_ecole_pourcent_insat',
       'Pistes

In [64]:
df_acc=df_complet[df_complet['Ecole_autre']==1]
proposition_acc=proposition(df_acc)
proposition_acc

,Nom question,Moyenne,Intitule question
5,Trottoir,0.494647,Des cheminements piétons (trottoirs) plus larg...
2,Vitesse,0.341905,Modérer la vitesse des véhicules motorisés dan...
0,Verbaliser,0.341132,Verbaliser davantage le stationnement sur les ...
13,Point_noir,0.287275,Traiter les points noirs et les coupures urbai...
1,Reserver,0.245006,Réserver l'usage des trottoirs aux déplacement...
11,Environnement,0.232204,Un environnement des cheminements piétons plus...
3,Compléter,0.231100,Constituer un réseau complet de cheminements p...
4,Priorite,0.183313,Davantage de lieux où les piétons sont priorit...
12,Agreable,0.153625,Rendre la marche plus agréable pour les piéton...
8,Bruit,0.103741,Des circulations piétonnes éloignées ou protég...


In [68]:
df_complet['Age_repondant'].unique()

array(['De 45 à 54 ans', 'De 65 à 74 ans', 'De 35 à 44 ans',
       'De 26 à 34 ans', '75 ans et plus', 'De 55 à 64 ans',
       'De 18 à 25 ans', 'Moins de 18 ans'], dtype=object)

In [70]:
df_age=df_complet[df_complet['Age_repondant']=='75 ans et plus']
proposition_age=proposition(df_age)
proposition_age.to_csv(r'C:\Users\sylva\OneDrive\Bureau\barometre\preconisation75.csv', index=False, sep=';')

In [75]:
df_complet['Age'].unique()

array(['45-65', '65-', '25-45', '-25'], dtype=object)

In [76]:
df_age=df_complet[df_complet['Age']=='-25']
proposition_age=proposition(df_age)
proposition_age.to_csv(r'C:\Users\sylva\OneDrive\Bureau\barometre\preconisation26.csv', index=False, sep=';')

,Nom question,Moyenne,Intitule question
5,Trottoir,0.382598,Des cheminements piétons (trottoirs) plus larg...
11,Environnement,0.361144,Un environnement des cheminements piétons plus...
13,Point_noir,0.266389,Traiter les points noirs et les coupures urbai...
12,Agreable,0.238975,Rendre la marche plus agréable pour les piéton...
0,Verbaliser,0.226460,Verbaliser davantage le stationnement sur les ...
3,Compléter,0.207986,Constituer un réseau complet de cheminements p...
1,Reserver,0.198451,Réserver l'usage des trottoirs aux déplacement...
4,Priorite,0.195471,Davantage de lieux où les piétons sont priorit...
2,Vitesse,0.194279,Modérer la vitesse des véhicules motorisés dan...
9,Air,0.137664,Une meilleure qualité de l'air


In [67]:
df_revenu=df_complet[df_complet['Plus revenu median']==0]
proposition_revenu=proposition(df_revenu)
proposition_revenu

,Nom question,Moyenne,Intitule question
5,Trottoir,0.408472,Des cheminements piétons (trottoirs) plus larg...
0,Verbaliser,0.300058,Verbaliser davantage le stationnement sur les ...
1,Reserver,0.293732,Réserver l'usage des trottoirs aux déplacement...
2,Vitesse,0.258118,Modérer la vitesse des véhicules motorisés dan...
11,Environnement,0.247997,Un environnement des cheminements piétons plus...
13,Point_noir,0.243627,Traiter les points noirs et les coupures urbai...
3,Compléter,0.238873,Constituer un réseau complet de cheminements p...
12,Agreable,0.232586,Rendre la marche plus agréable pour les piéton...
4,Priorite,0.160399,Davantage de lieux où les piétons sont priorit...
6,Communication,0.138432,Des actions de communication pour développer l...


In [33]:
df_prop_marseille=pd.DataFrame([i for i in range(15)])
liste_nom=[]
liste_valeur=[]
liste_pourcentage=[]
for i in prop :
    liste_nom.append(i)
    liste_valeur.append(np.mean(df_complet_marseille[i]))
    #liste_pourcentage.append(np.mean(df_complet[i+'_pourcent_insat']))
df_prop_marseille['Nom question']=np.array(liste_nom)
df_prop_marseille['Moyenne']=np.array(liste_valeur)
#df_moyenne['Pourcent']=np.array(liste_pourcentage)
df_prop_marseille=df_prop_marseille[['Nom question','Moyenne']]
df_prop_marseille

,Nom question,Moyenne
0,Verbaliser,0.458121
1,Reserver,0.372178
2,Vitesse,0.190823
3,Compléter,0.182083
4,Priorite,0.149308
5,Trottoir,0.544064
6,Communication,0.037145
7,Acces,0.041515
8,Bruit,0.111435
9,Air,0.149308


In [34]:
df_prop.to_csv(r'C:\Users\sylva\OneDrive\Bureau\barometre\preconisation28.csv', index=False, sep=';')

Base pmr

In [36]:
df['Lors_de_vos_deplacements'].unique()

array(["Un cycliste ou un utilisateur d'engin de déplacement personnel (vélo, trottinette, skate, roller…)",
       'Un marcheur, un piéton',
       'Un utilisateur des transports en commun',
       'Un utilisateur de véhicule motorisé (moto et autres deux roues, voitures à moteur à essence, gasoil...)',
       "Un utilisateur d'engin de déplacement personnel à assistance électrique (vélo, trottinette, skate électrique...)",
       'Une personne a mobilité réduite', nan], dtype=object)

In [42]:
df_pmr=pd.DataFrame([i for i in range(2)])
liste_nom=[]
liste_valeur_pmr=[]
liste_valeur=[]
var=['Marche_separe_circulation', 'Marche_separe_autres_mobilites']
df_data_pmr=df_complet[df_complet['Lors_de_vos_deplacements']=='Une personne a mobilité réduite']
for i in var :
    liste_nom.append(i)
    liste_valeur.append(np.mean(df_complet[i]))
    liste_valeur_pmr.append(np.mean(df_data_pmr[i]))
df_pmr['Nom question']=np.array(var)
df_pmr['Moyenne']=np.array(liste_valeur)
df_pmr['Moyenne_pmr']=np.array(liste_valeur_pmr)
df_pmr=df_pmr[['Nom question','Moyenne', 'Moyenne_pmr']]

In [47]:
df_pmr.to_excel(r'C:\Users\sylva\OneDrive\Bureau\barometre\pmr.xls', index=False)